# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [1]:
import pandas as pd
import numpy as np
from scipy.spatial.distance import pdist, squareform
from pprint import pprint


In [2]:
data = pd.read_csv('../data/customer_product_sales.csv')
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [9]:
products_by_client = data.groupby(['CustomerID', 'ProductName'])[['Quantity']].sum().reset_index()
products_by_client.head()

,CustomerID,ProductName,Quantity
0,33,Apricots - Dried,1
1,33,Assorted Desserts,1
2,33,Bandage - Flexible Neon,1
3,33,"Bar Mix - Pina Colada, 355 Ml",1
4,33,"Beans - Kidney, Canned",1


## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [10]:
pivot_table = pd.pivot_table(products_by_client, values = 'Quantity', index = 'ProductName', columns = 'CustomerID')
pivot_table = pivot_table.fillna(0)
pivot_table.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"Appetizer - Mini Egg Roll, Shrimp",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,25.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Appetizer - Mushroom Tart,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0
Appetizer - Sausage Rolls,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,25.0,25.0,25.0,0.0,25.0,0.0
Apricots - Dried,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
pivot_table.tail(5)

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Yeast Dry - Fermipan,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0
Yoghurt Tubes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0,0.0
"Yogurt - Blueberry, 175 Gr",0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,25.0,0.0,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0
Yogurt - French Vanilla,1.0,0.0,0.0,1.0,0.0,0.0,2.0,0.0,0.0,1.0,...,0.0,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0
Zucchini - Yellow,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
pivot_t =pd.pivot_table(data, index=['ProductName','CustomerID'], values= 'Quantity',fill_value=0)
pivot_t.head(5)


Quantity
ProductName               CustomerID          
Anchovy Paste - 56 G Tube 639                1
                          883                1
                          891                1
                          2566               1
                          2902               1

## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [14]:

customer_similarity = data.pivot_table(values='Quantity', columns='CustomerID', index='ProductName', aggfunc='sum').fillna(0)
distances = squareform(pdist(customer_similarity.T, 'euclidean'))
#distances

In [17]:
distances

array([[  0.        ,  11.91637529,  10.48808848, ..., 228.62851966,
        239.        , 229.77380181],
       [ 11.91637529,   0.        ,  11.74734012, ..., 228.01096465,
        239.03765394, 229.70415756],
       [ 10.48808848,  11.74734012,   0.        , ..., 228.08112592,
        238.26665734, 229.77380181],
       ...,
       [228.62851966, 228.01096465, 228.08112592, ...,   0.        ,
        304.13812651, 305.16389039],
       [239.        , 239.03765394, 238.26665734, ..., 304.13812651,
          0.        , 303.10889132],
       [229.77380181, 229.70415756, 229.77380181, ..., 305.16389039,
        303.10889132,   0.        ]])

In [5]:
#pivot_table_transpose = pivot_table.transpose()

#customer_similarity_matrix = pd.DataFrame(squareform(pdist(pivot_table_transpose, 'euclidean')),
                columns = pivot_table_transpose.index, index = pivot_table_transpose.index)

#customer_similarity_matrix.head()


CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,0.000000,11.916375,10.488088,11.224972,11.401754,11.090537,12.409674,11.045361,11.269428,11.489125,...,206.871941,213.180675,225.656819,198.232187,230.913404,220.501701,217.188858,228.628520,239.000000,229.773802
200,11.916375,0.000000,11.747340,12.083046,12.569805,12.288206,12.165525,12.083046,11.874342,12.000000,...,206.310446,212.635839,224.697575,197.139544,230.952376,220.202180,215.728997,228.010965,239.037654,229.704158
264,10.488088,11.747340,0.000000,11.489125,11.224972,11.445523,12.000000,11.401754,11.180340,11.747340,...,206.387984,212.946003,225.435135,197.600607,230.371439,219.136943,216.612557,228.081126,238.266657,229.773802
356,11.224972,12.083046,11.489125,0.000000,12.083046,11.789826,12.328828,11.135529,11.958261,12.165525,...,206.649462,213.082144,225.452878,197.494304,231.038958,219.952268,217.437347,228.098663,238.493186,229.464594
412,11.401754,12.569805,11.224972,12.083046,0.000000,11.704700,12.328828,11.135529,11.789826,11.747340,...,206.900942,211.679002,225.572605,197.630969,230.614397,219.733930,217.446545,227.997807,238.396728,228.927936


## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [20]:
customer33 = distances[33].sort_values(ascending=False)

top_5= customer33[1:6].index.to_list()

top_5

AttributeError: 'numpy.ndarray' object has no attribute 'sort_values'

In [18]:
list_top5 = list(customer_similarity_matrix[b33].sort_values(ascending=False).index)[1:6]

NameError: name 'b33' is not defined

In [7]:
list_top5

[97324, 95017, 92492, 91777, 92525]

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [8]:
records = df[df['CustomerID'].isin(list_top5)]
records.head()

NameError: name 'df' is not defined

## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [ ]:
records_agg = records.groupby('ProductName')[['Quantity']].sum().sort_values('Quantity', ascending=False)
records_agg.head()


In [ ]:
records_agg = records.groupby(['CustomerID', 'ProductName']).agg({'Quantity': sum}).sort_values(
    'Quantity', ascending=False)
records_agg.head()



## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

#### Merge the ranked products data frame with the customer product matrix on the ProductName field.

In [ ]:
products_similar_customers = records_agg.groupby('ProductName').agg({'Quantity': sum}).sort_values(
    'Quantity', ascending=False)
products_similar_customers.head()

#### Filter for records where the chosen customer has not purchased the product.


In [ ]:
not_purchased = pivot_table[pivot_table[33]==0]
not_purchased.head()

#### Show the top 5 results.

In [ ]:
products_not_purchased = not_purchased.merge(products_similar_customers, 
    left_on= 'ProductName', right_on= 'ProductName').sort_values('Quantity', ascending= False)
products = products_not_purchased[1:6].index.to_list()
products

## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

#### Create an empty dictionary that will hold the recommendations for all customers.

In [ ]:
recommendations = {}

#### Create a list of unique CustomerIDs to iterate over

In [ ]:
customers_ids = data['CustomerID'].unique()
customers_ids_list = customers_ids.tolist()
customers_ids_list

#### Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created

In [ ]:
recommendations = {}
customers_ind = data.index.tolist()
#customers_ids_list = customers_ids.tolist()
#customers_ids_list

for customers in customers_ind:
    list_top5 = list(customer_similarity_matrix[33].sort_values(ascending=False).index)[:5]
    records = df[df['CustomerID'].isin(list_top5)]
    records_agg = records.groupby(['CustomerID', 'ProductName']).agg({'Quantity': sum}).sort_values(
    'Quantity', ascending=False)
    products_similar_customers = records_agg.groupby('ProductName').agg({'Quantity': sum}).sort_values(
    'Quantity', ascending=False)
    not_purchased = pivot_table[pivot_table[33]==0]
    products_not_purchased = not_purchased.merge(products_similar_customers, 
    left_on= 'ProductName', right_on= 'ProductName').sort_values('Quantity', ascending= False)
    products = products_not_purchased[1:6].index.to_list()
    recommendations.update({customers:products})
        

In [ ]:
customers_ids = [data['CustomerID'].unique()]
customers_ids

In [ ]:
unique_custID = list(customers_ids['CustomerID'].unique())

In [ ]:
def recommendations_dict_for_customers(unique_custID):
    recommendations = {}
    for cust in unique_custID:
        top_5_cust = customer_similarities[cust].sort_values(ascending=False)[1:6].index
        df = df1.loc[top_5_cust]
        df = df.groupby(by='ProductName').agg({'Quantity':sum})
        df = df.sort_values(by='Quantity', ascending=False)
        df = df.merge(customer_product_matrix, on='ProductName')
        df = df[df[chosen_customer]==0]
        res = df['Quantity'].nlargest(5)
        products = list(res.index)
        recommendations[cust] = products
    return recommendations


In [ ]:
recommendations_dict_for_customers(unique_custID)

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [ ]:
results_df = pd.DataFrame(recommendations)
results_df.head()

In [ ]:
# Hago df desde diccionario y reinicio index:
results = pd.DataFrame.from_dict(recommendations, columns = ['CustomerID', 'Product1', 'Product2', 'Product3', 'Product4','Product5'], orient = 'index').reset_index()

# Cambio el nombre del index:
#results.index.names = ['CustomerID']
results.head()


In [ ]:
results2 = pd.DataFrame.from_dict(recommendations_for_everyone, orient='index').reset_index()
results2.columns = ['CustomerID','Prod_01', 'Prod_02', 'Prod_03', 'Prod_04', 'Prod_05']
results2.head()

## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.